In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
api_key = os.getenv("TOGETHER_API_KEY")

In [4]:
import os
import requests


url = "https://api.together.xyz/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Example message
messages = [
    {"role": "user", "content": "สวัสดี Typhoon ช่วยวิเคราะห์ข้อมูลหน่อย"}
]

response = requests.post(url, headers=headers, json={
    "model": "scb10x/scb10x-llama3-1-typhoon2-8b-instruct",
    "messages": messages
})

# Print the model's response
print(response.json()["choices"][0]["message"]["content"])


สวัสดีครับ! ผมยินดีที่จะช่วยวิเคราะห์ข้อมูลให้คุณครับ กรุณาแชร์ข้อมูลที่คุณต้องการให้ผมวิเคราะห์ และบอกว่าคุณต้องการให้ผมวิเคราะห์ในด้านใด เช่น สถิติ, การเปรียบเทียบ, หรือการวิเคราะห์เชิงลึกอื่น ๆ ครับ


In [23]:
system_message = "You are an advanced language model assisting in the analysis of public perception around the Faculty of Medicine, Ramathibodi Hospital, Mahidol University. You are given an Instagram post caption made by medical students, staff, patients, or the public, and you can use this information to weight the category of the caption. Your task is to analyze each caption and classify it into one or more predefined thematic categories and give a short reasoning. You respond strictly in this format, \"Category name(s): short reasoning\". The seven categories are  \"Hospital Service\" which includes all the caption that directly mention delivering a service or receiving a service at Ramathibodi Hospital or its related organization, \"Academic Experience\" of medical professional and student trainings  (e.g, teaching quality in both clinical and preclinical settings, research output, and academic program variety like dual degree program and surgical residency training), \"Student Engagement and Support\" (e.g., Student involvement in faculty decision making and representation, mental health support, extra-curricular activities, an student organizations and communities like Ramathibodi medical student committee), \"Career Outcomes\" (e.g., residency match rates, alumni stories, career support services), \"Prospect Student Admissions\" (e.g., TCAS medical school admission process, Ramathibodi open house and campus tours), \"Infrastructure\" (e.g., hospital buildings e.g., Queen Sirikit Medical Center, campus facilities i.e., Phayathai, CNMI, and research facilities), and \"Resources\" (e.g., medical equipment, technology integration, and hospital funding). If the caption does not fit with any group, respond \"Other\" and no category must be responded at the same time as \"Other\" . When you see hashtags in a sentence you can recognize it as instagram hashtags, and are able to use the hashtag to weight the content’s category." 

In [5]:
def typhoon_call(system_message, prompt):
    
    data = {
        "model": "scb10x/scb10x-llama3-1-typhoon2-8b-instruct",
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "top_p": 0.9,
        "repetition_penalty": 1.1,
    }

    response = requests.post(url, headers=headers, json=data)
    try:
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error in API response:", e)
        print("Full response:", response.text)
        return None

## Load CSV

In [6]:
import pandas as pd
df = pd.read_csv(r"C:\Users\monrada\Documents\Bridge\instagram\ig-category\20results.csv")
df

,caption
0,ครบ2ปีที่พ่อเข้ารับการรักษาครั้งใหญ่ #เราผ่านม...
1,รักษาสิวกับหมอครั้งแรก เริ่ม✌🏻 #จะรอดรึจะร่วง ...
2,ของแถมจากการไปหาหมอรักษาคีรอยด์ ... Fine Scan!...
3,อาคารใหม่ เหมือนเป็นอีกโรงพยาบาลเลย \n#รามาพรี...
4,โดนสั่งตรวจ! \n\n•เจาะ🩸ไป 6 หลอด! เลยถามคุณหมอ...
5,"#ผู้ป่วยนอก วงเงิน 100 ล้าน😱\n✅ ค่าห้อง 15,000..."
6,#รามาพรีเมี่ยม มืออาชีพ บริการไว ถูกใจป้าสุดๆ❤...
7,#บริจาคเลือด #บริจาคโลหิต #โรงพยาบาลรามาธิบดี ...
8,ไปหาหมอจริงๆนะ #เจอเพื่อนบ้าง #เจอคนที่ชื่นชอบ...
9,#รามาพรีเมี่ยม #หมอเก่งมาก มารักษาปลายเส้นประส...


In [24]:
results =[]
for index, row in df.iterrows():
    prompt_row = f"Cateogorize this into one or more of the seven categories, if it does not fit responds \'other'\. {row[0]}"
    result = typhoon_call(system_message, prompt_row)
    if result is not None:
        results.append({
            "response": result
        })
    else:
        print(f"Skipping prompt due to error: {prompt_row}")

print(results)

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\monrada\AppData\Local\Temp\ipykernel_2776\380661226.py:3: SyntaxWarning: invalid escape sequence '\.'
  prompt_row = f"Cateogorize this into one or more of the seven categories, if it does not fit responds \'other'\. {row[0]}"
C:\Users\monrada\AppData\Local\Temp\ipykernel_2776\380661226.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prompt_row = f"Cateogorize this into one or more of the seven categories, if it does not fit responds \'other'\. {row[0]}"


[{'response': "Hospital Service: This caption mentions a significant treatment experience for the author's father at Ramathibodi Hospital, indicating a direct interaction with the hospital's services."}, {'response': "Hospital Service: The caption mentions visiting a doctor for the first time to treat acne, indicating a direct interaction with the hospital's service."}, {'response': 'Hospital Service: The user received treatment for a skin condition called keratosis using fine scan technology at Ramathibodi Hospital. They also mentioned anesthesia administration during the procedure.'}, {'response': 'Category name(s): Infrastructure'}, {'response': 'Hospital Service: The captions describe a medical examination and consultation, indicating a service received at Ramathibodi Hospital.'}, {'response': 'Category name(s): Hospital Service'}, {'response': 'Hospital Service: The caption mentions receiving a service at Ramathibodi Hospital, specifically highlighting the professionalism and prom

In [25]:
df_result = pd.DataFrame(results)
df_complete = pd.concat([df,df_result],axis=1)
pd.set_option('display.max_colwidth', None)
df_complete

,caption,response
0,ครบ2ปีที่พ่อเข้ารับการรักษาครั้งใหญ่ #เราผ่านมาได้ราบรื่น #รามาพรีเมี่ยม Praewa Khoomboon Phimnapat Thammabut. บุญส่ง คุ้มบุญ,"Hospital Service: This caption mentions a significant treatment experience for the author's father at Ramathibodi Hospital, indicating a direct interaction with the hospital's services."
1,รักษาสิวกับหมอครั้งแรก เริ่ม✌🏻 #จะรอดรึจะร่วง ไม่ลองไม่รู้ #รามาพรีเมี่ยม #เลเซอร์ลดรอยสิว,"Hospital Service: The caption mentions visiting a doctor for the first time to treat acne, indicating a direct interaction with the hospital's service."
2,ของแถมจากการไปหาหมอรักษาคีรอยด์ ... Fine Scan!! [รอบนี้ขอนวดยาชาเอง รอบที่แล้วทำเลเซอร์ หมอใจดีกลัวไม่สวยยิงตรงที่ไม่ทายาชา อิชั้นน้ำตาเล็ดเลยจ้า]\n\nรอบนี้ขนาดทาเอง ... โอ้!! แม่เจ้าเหมือนหน้าจะไหม้ ใครนึกภาพไม่ออก ก็คิดซะว่าเหมือนกับปืนพ่นไฟมาจี้ ๆ ตรงหน้า พ่นไปเรื่อย ๆ อิชั้นก็ได้แต่สะกดจิตตัวเอง ท่องไป จะสวย จะสวย จะสวย!! ทำเสร็จหน้าแดงเป็นกวนอูเลยจ้า #finescan #5000 #รามาพรีเมี่ยม #ผู้หญิงอย่าหยุดสวย #แก่แล้วต้องบำรุง\n\nFine Scan เป็นเทคโนโลยีการยิงเลเซอร์ลำแสงแคบเป็นจุดเล็กๆ นับพันจุดบนผิวหนังต่อตารางเซนติเมตร เพื่อกระตุ้นการสร้างเนื้อเยื่อเผยผิวที่สุขภาพดีขึ้น โดยเลเซอร์มีขนาดเล็กกว่าเส้นผม ผ่านการควบคุมความลึกอย่างเที่ยงตรงสู่ชั้นโครงสร้างคอลลาเจนของผิว ซึ่งเลเซอร์สามารถส่งผ่านผิวหนังลงไปได้ลึกถึง 1500 ไมครอน จึงเป็นเลเซอร์ที่รักษาปัญหาผิวได้อย่างดีเยี่ยม👍🏻,Hospital Service: The user received treatment for a skin condition called keratosis using fine scan technology at Ramathibodi Hospital. They also mentioned anesthesia administration during the procedure.
3,อาคารใหม่ เหมือนเป็นอีกโรงพยาบาลเลย \n#รามาพรีเมี่ยม #ramapremium #ราคาก็พรีเมี่ยม #คุณหมอเก่งมาก #hospital #doctor #sick,Category name(s): Infrastructure
4,โดนสั่งตรวจ! \n\n•เจาะ🩸ไป 6 หลอด! เลยถามคุณหมอว่า ผลเลือดบอกไหมว่าผมจะรวยเมื่อไรครับ? 😚\n\n•คุณหมอ มองหน้า😊แล้วยิ้มให้ทีนึง และพูดเบาๆว่า ดูจากผลเลือดแล้ว… ”ปีหน้าคุณรวยนะ“ \n\n•สาธุ สิรออะไร🙏🙏🙏☺️\n\nกราบงามๆพี่เบล @belldec พี่แอม มา ณ โอกาสนี้ด้วยครับ 🙏❤️\n\n#BasPiriyakul \n#PiriyaInternational \n#รามา #โรงพยาบาลรามาธิบดี \n#รามาพรีเมี่ยม #Bangkok #BikeForLife,"Hospital Service: The captions describe a medical examination and consultation, indicating a service received at Ramathibodi Hospital."
5,"#ผู้ป่วยนอก วงเงิน 100 ล้าน😱\n✅ ค่าห้อง 15,000- บาท/คืน\n✅ คลอดบุตรเบิกได้ 200,000-400,000 บาท\n✅ มีวงเงินค่ารักษา 100 ล้านบาท / ปี\n✅ เบิกได้ทั้งผู้ป่วยนอก และผู้ป่วยใน\n✅ ฉีดวัคซีน ทันตกรรม และสายตา เบิกได้\n✅ รับค่าตรวจสุขภาพ สูงสุดถึง 20,000 บาท/ปี\n.\nอัพเดทใหม่ 🎉\nใช้ได้ทุกโรงพยาบาลในประเทศไทย\n✅ รพ.กรุงเทพ\n✅ รพ.สมิติเวช\n✅ รพ.พญาไท\n✅ รพ.บำรุงราษฎร์\n✅ รพ.ศิริราช ปิยมหาราชการุณย์\n✅ รพ.รามา พรีเมี่ยม\nและโรงพยาบาลชั้นนำในเครือข่ายอยุธยาแคร์ ทั่วประเทศมากว่า​ 600 แห่งทั่วประเทศ\n-------------\n👉สนใจสอบถามได้นะคะ\n👉หรือนัดเจอเพื่อคุยรายละเอียด\n🎊สอบถามรายละเอียด​เพิ่มเติม​พร้อมรับโปรโมชั่นพิเศษได้เลยค่ะ​ 🎁\n--------------\nติดต่อ คุณกุลลินี ศรีวิชัย (จุกจิก)\nตัวแทนประกันชีวิต บมจ.อลิอันซ์ อยุธยา ประกันชีวิต\n☎️ 093-2426616\n.\n..\n..\n#HealthCare #ประกันสุขภาพ #HealthInsurance #ตัวแทนประกันชีวิต #Allianz #อลิอันซ์อยุธยา #ประกันคลอดบุตร #ผู้ป่วยนอกจ่ายตามจริง\n#โรงพยาบาลกรุงเทพ #โรงพยาบาลสมิติเวช #โรงพยาบาลพญาไท #โรงพยาบาลเปาโลเมโมเรียล #โรงพยาบาลบีเอ็นเอช #โรงพยาบาลบำรุงราษฎร์ #รามาพรีเมี่ยม #โรงพยาบาลศิริราชปิยมหาราชการุณย์ #ตัวแทนอลิอันซ์",Category name(s): Hospital Service
6,#รามาพรีเมี่ยม มืออาชีพ บริการไว ถูกใจป้าสุดๆ❤️❤️🥰🥰,"Hospital Service: The caption mentions receiving a service at Ramathibodi Hospital, specifically highlighting the professionalism and promptness of the care received."
7,#บริจาคเลือด #บริจาคโลหิต #โรงพยาบาลรามาธิบดี #รามาพรีเมี่ยม \nสำหรับท่านที่สะดวกบริจาคเลือดค่ะ ^^\n\nขออนุญาตแจ้งข่าว \n\nเนื่องด้วยนายศุภกร วงษ์กุหลาบ\nนร.ชั้นม.5/3\nกำลังเตรียมจะผ่าตัดหลัง ที่โรงพยาบาลรามา (พรีเมี่ยม) วันที่12 มิถุนายน 2566 นี้\nแต่ทางรพ.อาจมีเลือดสำรองไม่เพียงพอ จึงขอบริจาคเลือดกรุ๊ป A \n(*)ผู้บริจาคจะเป็นเลือดกรุ๊ปใดก็ได้ สามารถนำมาแลกเป็นกรุ๊ป A ได้ \n(*) สามารถบริจาคได้ที่ โรงพยาบาลรามา (เขียนกำกับว่าเพื่อนายศุภกร วงษ์กุ